In [ ]:
# Importing necessary packages
import spacy
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
nlp = spacy.load('en_core_web_sm')

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import TensorDataset, DataLoader, random_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
trainPath = "/content/drive/MyDrive/IISc/DLNLP/Assignment2/train.csv"
trainData = pd.read_csv(trainPath)
trainData = trainData.values.tolist()

In [ ]:
def posTo1Negto0(tempStr):
  if(tempStr == "positive"):
    return 1
  return 0

trainData = np.array(trainData)[:500]
X_sentences = trainData[:,0]
y = [posTo1Negto0(x) for x in trainData[:,1]]

AttributeError: ignored

In [ ]:
print(trainData.shape)
print(X_sentences[4])
print(y[4])

(500, 2)
Petter Mattei's "Love in the Time of Money" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />This being a variation on the Arthur Schnitzler's play about the same theme, the director transfers the action to the present time New York where all these different characters meet and connect. Each one is connected in one way, or another to the next person, but no one seems to know the previous point of contact. Stylishly, the film has a sophisticated luxurious look. We are taken to see how these people live and the world they live in their own habitat.<br /><br />The only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits. A big city is not exactly the best place in which human relations find sincere fulfillment, as one discerns is the ca

Functions for sentence preprocessing

In [ ]:
def remPunc(text):
  if type(text) == str:
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', '', text.lower())
    return text

def refineSentence(tempSentence):
  tokens = nlp(str(tempSentence))
  filtered = []
  for token in tokens:
    if (token.is_stop == False):
      textAfterRemPunc = remPunc(token.lemma_)
      if(textAfterRemPunc != ''):
        filtered.append(textAfterRemPunc)
  return filtered 

PreProcessing the Train Data

In [ ]:
preProccessedList = [refineSentence(sentence) for sentence in X_sentences]

In [ ]:
print(preProccessedList[0])

['reviewer', 'mention', 'watch', '1', 'oz', 'episode', 'hook', 'right', 'exactly', 'happen', 'mebr', 'br', 'the', 'thing', 'strike', 'oz', 'brutality', 'unflinche', 'scene', 'violence', 'set', 'right', 'word', 'trust', 'faint', 'hearted', 'timid', 'pull', 'punch', 'regard', 'drug', 'sex', 'violence', 'hardcore', 'classic', 'use', 'wordbr', 'br', 'it', 'call', 'oz', 'nickname', 'give', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'focus', 'mainly', 'emerald', 'city', 'experimental', 'section', 'prison', 'cell', 'glass', 'front', 'face', 'inward', 'privacy', 'high', 'agenda', 'em', 'city', 'home', 'aryans', 'muslims', 'gangsta', 'latinos', 'christians', 'italians', 'irish', 'scuffle', 'death', 'stare', 'dodgy', 'dealing', 'shady', 'agreement', 'far', 'awaybr', 'br', 'i', 'main', 'appeal', 'fact', 'go', 'show', 'dare', 'forget', 'pretty', 'picture', 'paint', 'mainstream', 'audience', 'forget', 'charm', 'forget', 'romance', 'oz', 'mess', 'episode', 'see', 'strike', 'nasty', 'su

Getting GloVe Embeddings

In [ ]:
embeddingsDict = {}
dim = 300
with open("/content/drive/MyDrive/IISc/DLNLP/Assignment1/glove.6B.300d.txt", 'r', encoding="utf-8") as wordEmbeddings:
  for line in wordEmbeddings:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    embeddingsDict[word]=vector

Function to convert refined sentence to average sentence vector

In [ ]:
def sentenceToWordEmbeddings(preProccessedList):
  preProccessedListVectors = []
  for preProcessedWordList in preProccessedList:
    tempList = []
    for word in preProcessedWordList:
      if(word in embeddingsDict):
        tempList.append(embeddingsDict[word])
    preProccessedListVectors.append(np.array(tempList, dtype = float))
  return preProccessedListVectors

Training Data to wordsVector Matrix

In [ ]:
wordsVector = sentenceToWordEmbeddings(preProccessedList)

In [ ]:
wordsVector = torch.tensor(wordsVector, dtype = float)
print(wordsVector[0].shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  """Entry point for launching an IPython kernel.


ValueError: ignored

In [ ]:
class LSTM(nn.Module):
  def __init__(self, input_size, num_layers, hidden_size, num_classes):
    super(LSTM, self).__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    
    self.LSTM = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers = num_layers, bias = True, batch_first = False, dropout = 0, bidirectional = False )

    self.fc = nn.Linear(hidden_size, num_classes)
    self.softmax=nn.Softmax(dim=1)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to_device

    out, _ = self.LSTM(x, h0)
    out = out[:,-1,:]
    out = self.fc(out)
    return out

In [ ]:
class SA_LSTM:
  def __init__(self, input_size, num_layers, hidden_size, num_classes, epochs,lr):
    self.input_size = input_size
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    self.num_classes = num_classes
    self.epochs = epochs
    
    self.model = LSTM(input_size, num_layers, hidden_size, num_classes)

    self.cross_entropy_loss = nn.CrossEntropyLoss()
    self.optimizer = optim.SGD(self.NNobj.parameters(), lr = lr)
    
    self.trainLossList = []
    self.validLossList = []
    self.trainLossListPlotting = []
    self.validLossListPlotting = []

  def fit(self, train_data, valid_data):
    for i in range(self.epochs):
      self.trainLossList = []
      self.validLossList = []
      accuracyListTrain = []
      for batch in train_data:
        self.model.train()                         #does dropout
        x, y = batch
        # b = x.size(0)
        # x = x.view(b, -1)

        forwardValue = self.model(x)
        costFunction_J = self.cross_entropy_loss(forwardValue, y)
        self.NNobj.zero_grad()
        costFunction_J.backward()
        self.optimizer.step()
        self.trainLossList.append(costFunction_J.item())
        accuracyListTrain.append(y.eq(forwardValue.detach().argmax(dim = 1)).float().mean())
          
      accuracyListVal = []
      for batch in valid_data:
        self.NNobj.eval()                           ##stops dropout
        x, y = batch
        with torch.no_grad():
          forwardValue = self.model(x)
          costFunction_J = self.cross_entropy_loss(forwardValue, y)
        self.validLossList.append(costFunction_J.item())
        accuracyListVal.append(y.eq(forwardValue.detach().argmax(dim = 1)).float().mean())
      
      self.trainLossListPlotting.append(torch.tensor(self.trainLossList).mean())
      self.validLossListPlotting.append(torch.tensor(self.validLossList).mean())
      accuracyTrain = torch.tensor(accuracyListTrain).mean() * 100
      accuracyVal = torch.tensor(accuracyListVal).mean() * 100

      print('At Epoch Number: ' + str(i+1) +'; Train Loss= ' + str("{:.2f}".format(torch.tensor(self.trainLossList).mean()))+'; Validation Loss= ' + str("{:.2f}".format(torch.tensor(self.validLossList).mean())) + "; " + "Train Accuracy = ", "{:.2f}".format(accuracyTrain), "%" + "; " + "Validation Accuracy = ", "{:.2f}".format(accuracyVal), "%")

  def predict(self,testData):
    accuracyListPred = []
    for batch in testData:
      x, y = batch

      with torch.no_grad():
        forwardValue = self.NNobj(x)
          
      accuracyListPred.append(y.eq(forwardValue.detach().argmax(dim = 1)).float().mean())
    accuracy = torch.tensor(accuracyListPred).mean() * 100
    print('\033[1m' + "Review Classification Accuracy on Test Data = ", "{:.2f}".format(accuracy), " %" + '\033[0m' )

  def plotLossCurve(self, flag):
    x = [(i+1) for i in range(self.epochs)]
    plt.xlabel('#Epoch')
    plt.ylabel('Loss')
    tempStr = "Loss curve for "
    if(flag == 0):
        tempStr += "Train Data"
        plt.plot(x,self.trainLossListPlotting)
    else:
        tempStr += "Validation Data"
        plt.plot(x,self.validLossListPlotting)
    plt.title(tempStr)
    plt.show()


In [ ]:
# HyperParameters
batch_size = 32
epochs = 10
hidden_size = 512
input_size = dim
seq_size = dim
num_classes = 2
num_layers = 2
lr = 1e-2

In [ ]:
obj = SA_LSTM(input_size, num_layers, hidden_size, num_classes, epochs,lr)
obj.fit(train_dataloader, valid_dataloader)
obj.plotLossCurve(0)
obj.plotLossCurve(1)

In [ ]:
obj.predict(test_dataloader)